In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import matplotlib.colors as clr
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw

Giulia Menichetti menicgiulia@gmail.com

# Import Data for Experimental Outcomes

In [2]:
specifics='Jaccard_Binary_8192b_R3'
#viz_data = pd.read_pickle('../data/Broad_Outcomes_Keys_with_fingerprint_embedding'+specifics+'.pkl')
viz_data = pd.read_csv('../data/data3Dspace.csv')

In [3]:
viz_data

,pert_iname,truncated_broadid,rdkit_smiles,rdkit_InChIKey,Outcome,Molecule,coord1,coord2,coord3
0,(R)-(-)-apomorphine,K76022557,CN1CCc2cccc3c2[C@H]1Cc1ccc(O)c(O)c1-3,VMWNQDUVQKEIOC-CYBMUJFWSA-N,No-Effect,"<img data-content=""rdkit/molecule"" src=""data:i...",2.626414,10.750898,9.543590
1,(R)-(-)-rolipram,K75516118,COc1ccc([C@@H]2CNC(=O)C2)cc1OC1CCCC1,HJORMJIFDVBMOB-LBPRGKRZSA-N,No-Effect,"<img data-content=""rdkit/molecule"" src=""data:i...",2.252012,11.441395,8.036580
2,(R)-baclofen,K62353271,NC[C@H](CC(=O)O)c1ccc(Cl)cc1,KPYSYYIEGFHWSV-QMMMGPOBSA-N,No-Effect,"<img data-content=""rdkit/molecule"" src=""data:i...",2.258490,8.440589,8.369008
3,(S)-(+)-rolipram,K65856711,COc1ccc([C@H]2CNC(=O)C2)cc1OC1CCCC1,HJORMJIFDVBMOB-GFCCVEGCSA-N,No-Effect,"<img data-content=""rdkit/molecule"" src=""data:i...",2.260489,11.457537,8.041487
4,"1,12-Besm",K04603573,CCNCCCNCCCCNCCCNCC,PGMIWVSHZMWSSI-UHFFFAOYSA-N,Very Weak (35%),"<img data-content=""rdkit/molecule"" src=""data:i...",3.663214,5.835912,8.038438
...,...,...,...,...,...,...,...,...,...
6704,zotarolimus,K46843573,CO[C@H]1C[C@@H]2CC[C@@H](C)[C@@](O)(O2)C(=O)C(...,CGTADGCBEXYWNE-JUKNQOCSSA-N,No-Effect,"<img data-content=""rdkit/molecule"" src=""data:i...",8.568832,8.443657,10.078921
6705,zotepine,K64557645,CN(C)CCOC1=Cc2ccccc2Sc2ccc(Cl)cc21,HDOZVRUNCMBHFH-UHFFFAOYSA-N,Low (25%),"<img data-content=""rdkit/molecule"" src=""data:i...",0.361577,9.902718,10.399397
6706,zoxazolamine,K66353228,Nc1nc2cc(Cl)ccc2o1,YGCODSQDUUUKIV-UHFFFAOYSA-N,Very Weak (35%),"<img data-content=""rdkit/molecule"" src=""data:i...",0.445708,9.268176,7.567175
6707,zuclopenthixol,K28761384,OCCN1CCN(CC/C=C2/c3ccccc3Sc3ccc(Cl)cc32)CC1,WFPIAZLQTJBIFN-DVZOWYKESA-N,No-Effect,"<img data-content=""rdkit/molecule"" src=""data:i...",0.380496,9.939382,10.405709


# Colormap for Experimental Outcomes

In [4]:
# colormap
cmapdict={'No-Effect': (179/255,226/255,205/255), 
'Low (25%)': (102/255,194/255,165/255),
'Very Weak (35%)': (27/255,158/255,119/255),          
'Weak (50%)': (231/255,41/255,138/255),          
'Strong (80%)': (117/255,112/255,179/255),          
'Cyto':(217/255,95/255,2/255)
}
ordered_classes = list(cmapdict.keys())


# colormap for Plotly
cmapdictplotly={k:'rgb'+str(tuple([v*255 for v in cmapdict[k]])) for k in ordered_classes}
alphadict={'No-Effect':0.1, 
'Cyto':0.3,
'Low (25%)': 0.3,
'Very Weak (35%)': 0.3,          
'Weak (50%)': 0.8,   
'Strong (80%)':0.8}

# HTML file Drug 3D Space with Experimental Classes

In [5]:
def make_label(row, name='pert_iname'):
    return '<br>Name: ' + row[name] \
        + '<br>Outcome: ' + row['Outcome'] \
        + '<br>BroadId: ' + row['truncated_broadid'] \
        + '<br>Smile: ' + row['rdkit_smiles'] \
        + '<br>InChiKey: ' + row['rdkit_InChIKey']

traces = []


for c in ordered_classes:
    traces.append( go.Scatter3d(
        x=viz_data[viz_data['Outcome'] == c].coord1.tolist(),
        y=viz_data[viz_data['Outcome'] == c].coord2.tolist(),
        z=viz_data[viz_data['Outcome'] == c].coord3.tolist(),
        mode='markers',
        marker=dict(
            opacity=alphadict[c],
            size=3,
            color=cmapdictplotly[c]
        ),
        name=c,
        text=viz_data[viz_data['Outcome'] == c].apply(make_label, axis=1),
        hoverinfo='text'
    ))
    
layout = go.Layout(
    title=specifics
)

fig = go.Figure(data=traces, layout=layout)
plot(fig, filename = '../figs/'+specifics+'.html', auto_open=True)

'../figs/Jaccard_Binary_8192b_R3.html'

# Import Data for ATC Codes

In [6]:
drugclassification=pd.read_csv('../data/drug-enrichment-Strong (80%) Weak (50%)-for3Dspace.tsv', sep='\t')

# Colormap for ATC codes

In [7]:
# colormap drug classes
listcolors=['rgb'+str(tuple([k*255 for k in n])) for n in (plt.get_cmap('Accent').colors[:-1]+plt.get_cmap('Dark2').colors[:-1]+plt.get_cmap('Set1').colors[:-1]+plt.get_cmap('Set2').colors[:-1])]
cmapdictplotlyDC={drugclassification['Chemical Group'][k]: listcolors[k] for k in range(len(drugclassification['Chemical Group']))}

# HTML file Drug 3D Space with enriched ATC codes

In [8]:
specifics='Enriched_Drug_Classes'
traces = []

# Strong and Weak Drugs
traces.append( go.Scatter3d(
    x=viz_data[viz_data['Outcome'].isin(['Strong (80%)', 'Weak (50%)'])].coord1.tolist(),
    y=viz_data[viz_data['Outcome'].isin(['Strong (80%)', 'Weak (50%)'])].coord2.tolist(),
    z=viz_data[viz_data['Outcome'].isin(['Strong (80%)', 'Weak (50%)'])].coord3.tolist(),
    mode='markers',
    marker=dict(
    opacity=0.3,
    size=3,
    color='rgb(72.0, 72.0, 72.0)'
        ),
    name="Strong (80%) & Weak (50%)",
    text=viz_data[viz_data['Outcome'].isin(['Strong (80%)', 'Weak (50%)'])].apply(make_label, axis=1),
    hoverinfo='text'
    ))


count=0
for c in drugclassification['Chemical Group']:
    dfsel=viz_data[viz_data['truncated_broadid'].isin(drugclassification['Broad IDs'].iloc[count].split(','))]
    
    print('%%%%%%%%%%%%%%%%%%')
    print(c)
    print('Number of Drugs')
    print(dfsel.shape[0])
    print(drugclassification['Broad IDs'].iloc[count].split(','))
    print('%%%%%%%%%%%%%%%%%%')
    
    count+=1
    
    traces.append( go.Scatter3d(
        x=dfsel.coord1.tolist(),
        y=dfsel.coord2.tolist(),
        z=dfsel.coord3.tolist(),
        mode='markers',
        marker=dict(
            opacity=0.3,
            size=10,
            color=cmapdictplotlyDC[c]
        ),
        name=c,
        text=dfsel.apply(make_label, axis=1),
        hoverinfo='text'
    ))
    
layout = go.Layout(
    title=specifics
)

fig = go.Figure(data=traces, layout=layout)
plot(fig, filename = '../figs/'+specifics+'.html', auto_open=True)

%%%%%%%%%%%%%%%%%%
Benzothiazines
Number of Drugs
11
['K37814297', 'K10995081', 'K89997465', 'A84481105', 'K35559145', 'K54770538', 'K19352500', 'K06980535', 'K69600043', 'K89732114', 'K08619574']
%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%
Phenothiazines
Number of Drugs
10
['K37814297', 'K10995081', 'K89997465', 'A84481105', 'K35559145', 'K19352500', 'K06980535', 'K69600043', 'K89732114', 'K08619574']
%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%
Phenothiazines
Number of Drugs
10
['K37814297', 'K10995081', 'K89997465', 'A84481105', 'K35559145', 'K19352500', 'K06980535', 'K69600043', 'K89732114', 'K08619574']
%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%
Antipsychotic Agents (First Generation [Typical])
Number of Drugs
8
['K10995081', 'K89997465', 'A84481105', 'K35559145', 'K19352500', 'K06980535', 'K89732114', 'K08619574']
%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%
Phenothiazines
Number of Drugs
10
['K37814297', 'K10995081', 'K89997465', 'A84481105', 'K35559145', 'K19352500', 'K06980535', 'K69600043', 'K8973211

'../figs/Enriched_Drug_Classes.html'